# base_line 모델(roberta-base)의 선정 및 f1_score 

Team LostCow의 roberta모델을 기준으로 함

출처: https://github.com/l-yohai/KLUE

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 15.0 MB/s 
     |████████████████████████████████| 895 kB 68.4 MB/s 
     |████████████████████████████████| 596 kB 56.2 MB/s 
     |████████████████████████████████| 6.5 MB 66.9 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 14.2 MB/s 
     |████████████████████████████████| 1.1 MB 67.0 MB/s 
     |████████████████████████████████| 134 kB 73.5 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
     |████████████████████████████████| 127 kB 59.0 MB/s 
     |████████████████████████████████| 144 kB 57.6 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 271 kB 77.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [6]:
import tarfile

#훈련 데이터셋 다운로드 및 압축풀기

!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

tar_bz2_file = tarfile.open("/content/klue-sts-v1.1.tar.gz")
tar_bz2_file.extractall(path="/content")
tar_bz2_file.close()

In [ ]:
import torch

#GPU 사용 여부 확인 및 name 확인

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

In [7]:
import json

#train 데이터셋을 train,valid 데이터셋으로 split

def read_json(file_path):
    with open(file_path) as f:
        return json.load(f)

data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_train.json')

train_length=int(len(data)*0.9)
train=data[:train_length]
vaild=data[train_length:]

print('data_set:',len(data))
print('train_set:',len(train),', valid_set:',len(vaild))

with open('train_split.json','w') as f:
  json.dump(train,f)
with open('valid_split.json','w') as f:
  json.dump(vaild,f)

data_set: 11668
train_set: 10501 , valid_set: 1167


In [8]:
!git clone https://github.com/l-yohai/KLUE #base line 모델 불러오기

Cloning into 'KLUE'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 313 (delta 52), reused 101 (delta 29), pack-reused 179
Receiving objects: 100% (313/313), 5.79 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [10]:
# base line 모델 학습
!python /content/KLUE/sts/train.py --data_dir /content --model_name_or_path "klue/roberta-base" --train_filename "train_split.json" --valid_filename "valid_split.json"

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForStsRegression: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForStsRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForStsRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForStsRegression were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'label_classifier.linear.bias', 'sentence_fc_layer.linear.bias', 'dens

In [21]:
mv /content/klue-sts-v1.1.tar.gz /content/model/klue-sts-v1.1.tar.gz

In [17]:
#학습한 base line 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/KLUE/sts/inference.py --test_filename "/content/klue-sts-v1.1/klue-sts-v1.1_dev.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [19]:
import pandas as pd

#Regression pred 를 binary-label pred로 변환

df_pred = pd.read_csv('/content/output.csv',header=None)
df_pred=(df_pred>2.5).astype(int)

In [24]:
#valid의 label 값을 추출

valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]

In [36]:
from sklearn.metrics import classification_report,f1_score

#혼돈 메트릭스 및 f1_score 시각화

print(classification_report(valid_label, df_pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, df_pred))

              precision    recall  f1-score   support

           0       0.99      0.69      0.81       299
           1       0.70      0.99      0.82       220

    accuracy                           0.82       519
   macro avg       0.85      0.84      0.82       519
weighted avg       0.87      0.82      0.82       519

base_line(roberta-base) f1_score: 0.8226415094339622
